In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('./data/train_cleaned.csv')
test = pd.read_csv('./data/test_cleaned.csv')

In [3]:
max_features = 30000
maxlen = 100
embed_size = 300
EMBEDDING_FILE = '/media/radoslav/ce763dbf-b2a6-4110-960f-2ef10c8c6bde/MachineLearning/crawl-300d-2M.vec'

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['comment_text'].append(test['comment_text']))

In [6]:
def get_features(comments):
    seq = tokenizer.texts_to_sequences(comments)
    seq = pad_sequences(seq, maxlen=maxlen)
    return seq

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_fold, val_fold = train_test_split(train, train_size=0.90 ,random_state=1234)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
train_seq = get_features(comments=train_fold['comment_text'])

In [10]:
val_seq = get_features(comments=val_fold['comment_text'])

In [11]:
import numpy as np
from util import labels, RocAucEvaluation, get_fastext_embedding

In [12]:
embedding_matrix = get_fastext_embedding(EMBEDDING_FILE, embed_size, tokenizer.word_index, max_features)

In [13]:
from keras.layers import Input, Bidirectional, CuDNNGRU, Dense, Dropout, Embedding, GlobalAveragePooling1D,GlobalMaxPooling1D,Concatenate, SpatialDropout1D
from keras.models import Model

In [14]:
def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = Concatenate()([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
    

In [15]:
model = get_model()

In [16]:
roc_auc = RocAucEvaluation(validation_data=(val_seq, val_fold[labels].values))

In [17]:
model.fit(train_seq, train_fold[labels].values,
          validation_data=(val_seq, val_fold[labels].values),
          epochs=2,
          batch_size=64,
          callbacks=[roc_auc])

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143552/143613 [============================>.] - ETA: 0s - loss: 0.0541 - acc: 0.9810
 ROC-AUC - epoch: 1 - score: 0.983949 

143613/143613 [==============================] - 76s 529us/step - loss: 0.0541 - acc: 0.9810 - val_loss: 0.0427 - val_acc: 0.9839
Epoch 2/2
143552/143613 [============================>.] - ETA: 0s - loss: 0.0384 - acc: 0.9850
 ROC-AUC - epoch: 2 - score: 0.982877 

143613/143613 [==============================] - 75s 522us/step - loss: 0.0384 - acc: 0.9850 - val_loss: 0.0431 - val_acc: 0.9838


In [18]:
test_seq = get_features(test['comment_text'])

In [19]:
preds = model.predict(test_seq)

In [20]:
subm = pd.DataFrame(preds, columns=labels, index=test['id'])

In [21]:
subm.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
00001cee341fdb12,0.996313,0.375589,0.988004,0.044333,0.958232,0.103217
0000247867823ef7,0.000225,0.000010,0.000083,0.000004,0.000079,0.000007
00013b17ad220c46,0.001614,0.000055,0.000453,0.000013,0.000519,0.000024
00017563c3f7919a,0.000538,0.000017,0.000234,0.000049,0.000353,0.000006
00017695ad8997eb,0.006324,0.000035,0.000533,0.000034,0.000437,0.000018


In [22]:
subm.to_csv('./submission-tmp/pooled_gru_fastext.csv')